<h1>Levaraged etfs and their drag</h1>

Levaraged ETFs like TQQQ, are popular with investors because they provide levarage. A $3 \times$ long ETF like TQQQ, provides $3$ times the daily return compared to the underlying QQQ. Similarly an inverse $3 \times$ ETF like SQQQ provides $-3$ times the daily return of the underlying QQQ. 

For most investors it is not recommended to buy and hold these ETFs over the long term. The reason given is that they suffer a drag and some investors argue that there is a structural problem with these ETFs. Reasons for the drag are generally attributed to the variance and the rebalance drag. The rebalance drag occurs due to the need for rebalancing the derivatives used in structuring of these ETFs. I will explain in this post that levaraged ETFs on indices such as S&P 500 (SPX) and Nasdaq-100 (NDX), do not suffer from rebalance drag. The only source of drag is the variance drag. 



I will illustrate the argument by followig two pairs of unleveraged vs levaraged ETFs:




 <ul>
  <li>SPY vs SPXL (ETFs on SPX)</li>
  <li>QQQ vs TQQQ (ETFs on NDX)</li>

</ul>  




<h1>Mathematics of stock prices</h1>

In mathematical finance and economics literature stock prices are modeled via Geometric Brownian Motion. This is described by the following equation. Let $S_t$ denote the price of a stock at time $t$.

$$dS_t = \mu dt + \sigma dW_t, $$

where $\mu$ is the average return of the stock and $\sigma$ is the standard deviation of the returns. $W_t$ is the standard Brownian motion with mean $0$ and variance $t$.

This type of an equation is called a stochastic differential equation. Intuitively, the equation says change in the price of stock is due to two components, 
<ul>
  <li>Deterministic component, which should be average return of the stock over the interval of time. This is precisely $\mu dt$.</li>
  <li>Random component, which should be thought of as noise. This is precisely $\sigma dW_t$. </li>

</ul> 

Let us now see this in action. 

In [56]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance
import datetime as dt
from datetime import date, timedelta, time
from datetime import datetime

import plotly

In [57]:
#Fetching the historical stock price data for unlevaraged etfs: SPY and QQQ
spy_info = yfinance.Ticker("SPY")
qqq_info = yfinance.Ticker("QQQ")


#Date range chosen for the historical data is 2010-02-12 to 2025-04-13. This is due to the fact that one of the levaraged ETFs TQQQ only started trading on 2010-02-12.
start_date = "2010-02-12"
end_date = "2025-04-13"

number_of_years = (dt.datetime.strptime(end_date,"%Y-%m-%d") - dt.datetime.strptime(start_date, "%Y-%m-%d"))/timedelta(days=365)
                 
spy_history = spy_info.history(start = start_date , end = end_date )
qqq_history = qqq_info.history(start = start_date, end = end_date)

spy_history.reset_index(inplace = True)
qqq_history.reset_index(inplace = True)


In [58]:
spy_history.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
0,2010-02-12 00:00:00-05:00,80.970301,81.810352,80.607039,81.764946,304622100,0.0,0.0,0.0
1,2010-02-16 00:00:00-05:00,82.385537,83.134770,81.598462,83.051521,159317500,0.0,0.0,0.0
2,2010-02-17 00:00:00-05:00,83.452640,83.558598,83.051535,83.445076,168845100,0.0,0.0,0.0
3,2010-02-18 00:00:00-05:00,83.308856,84.111066,83.278584,83.937004,193708600,0.0,0.0,0.0
4,2010-02-19 00:00:00-05:00,83.717511,84.436471,83.520741,84.111046,222684900,0.0,0.0,0.0


In [59]:
qqq_history.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
0,2010-02-12 00:00:00-05:00,37.747225,38.270525,37.642566,38.165863,93938500,0.0,0.0,0.0
1,2010-02-16 00:00:00-05:00,38.444945,38.680426,38.244345,38.654263,83861400,0.0,0.0,0.0
2,2010-02-17 00:00:00-05:00,38.828718,38.872326,38.601954,38.872326,82691700,0.0,0.0,0.0
3,2010-02-18 00:00:00-05:00,38.846140,39.186283,38.767645,39.116508,65752600,0.0,0.0,0.0
4,2010-02-19 00:00:00-05:00,39.011842,39.290934,38.915903,39.099060,80416500,0.0,0.0,0.0


In [60]:
qqq_history.tail()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
3810,2025-04-07 00:00:00-04:00,408.660004,443.140015,402.390015,423.690002,161557000,0.0,0.0,0.0
3811,2025-04-08 00:00:00-04:00,438.160004,443.140015,409.790009,416.059998,101248100,0.0,0.0,0.0
3812,2025-04-09 00:00:00-04:00,415.570007,467.829987,415.429993,466.000000,142876900,0.0,0.0,0.0
3813,2025-04-10 00:00:00-04:00,453.559998,455.589996,432.630005,446.179993,108384100,0.0,0.0,0.0
3814,2025-04-11 00:00:00-04:00,444.649994,455.790009,441.329987,454.399994,52483800,0.0,0.0,0.0


In [61]:
#Compute the daily return percentage for SPY and QQQ
dailyret_spy = spy_history.loc[:, "Close"].pct_change()
dailyret_qqq = qqq_history.loc[:,"Close"].pct_change()

#Compute the average return for SPY and QQQ
mean_spy = np.mean(dailyret_spy)
mean_qqq = np.mean(dailyret_qqq)

#Since we computed the daily returns based on trading days, to annualize the returns we multiply by 252, for the trading days 
print("Average annual return over the last ", number_of_years, "years for SPY is:", mean_spy*252*100,"%")
print("Average annual return over the last ", number_of_years," years for QQQ is:", mean_qqq*252*100,"%")

Average annual return over the last  15.175342465753424 years for SPY is: 13.91656103307539 %
Average annual return over the last  15.175342465753424  years for QQQ is: 18.543826238712466 %


In [62]:
#Compute the standard deviation of returns and then annualize them.

sigma_spy = np.std(dailyret_spy)*np.sqrt(252)
sigma_qqq = np.std(dailyret_qqq)*np.sqrt(252)

#Usually the standard deviation is annualized and shown as percentage. This is also called the volatility of the stock.

print("Annual volatility over last", number_of_years, " years for SPY is:", sigma_spy*100,"% per year.")
print("Annual volatility over last", number_of_years, "  years for QQQ is:", sigma_qqq*100,"% per year.")



Annual volatility over last 15.175342465753424  years for SPY is: 17.385696266786667 % per year.
Annual volatility over last 15.175342465753424   years for QQQ is: 20.823320734444195 % per year.


Investors are usually interested in compound returns over a period of time. If one assumes continuous compounding, the rate of return over a time period is given by $\ln \frac{S_t}{S_0}$, where $S_0$ is the price of the stock on the day of initial investment and $S_t$ is the current price of stock. We can compute this using the following code.



In [63]:
#computing the annualized compound interest over the 15 years of data.
compunded_spy = np.log(spy_history["Close"][len(spy_history)-1]/spy_history["Close"][0])/number_of_years
compunded_qqq = np.log(qqq_history["Close"][len(qqq_history)-1]/qqq_history["Close"][0])/number_of_years

print("Compounded annual return over the last", number_of_years, " years for SPY is:", compunded_spy*100, "% per year.")
print("Compounded annual return over the last", number_of_years, " years for QQQ is:", compunded_qqq*100, "% per year.")

Compounded annual return over the last 15.175342465753424  years for SPY is: 12.365024798261198 % per year.
Compounded annual return over the last 15.175342465753424  years for QQQ is: 16.322770845248808 % per year.


<h1>Mathematical model for compounding </h1>

As one can see from above the compounded return differs from the average annual return substantially. This is mathematically modeled by the following equation

$$d \ln S_t = (\mu - \frac{1}{2} \sigma^2) dt + \sigma dW_t,$$

where we assume the equation above for the stock price $S_t$. This equation is due to Ito's formula. Ito's formula is foundation of modern mathematical finance. Using our earlier computation of the average return and the standard deviation of the stocks we can compute as follows.

In [64]:
#Computing compounded returns using Ito formula.
compounded_spy_ito = mean_spy*252 - ((sigma_spy*sigma_spy)/2)
compounded_qqq_ito = mean_qqq*252 - ((sigma_qqq*sigma_qqq)/2)
print("Compounded annual return over the last", number_of_years, "  years for SPY using Ito formula is:", compounded_spy_ito*100, "% per year.")
print("Compounded annual return over the last", number_of_years, "  years for QQQ using Ito formula is:", compounded_qqq_ito*100, "% per year.")

Compounded annual return over the last 15.175342465753424   years for SPY using Ito formula is: 12.40524885967059 % per year.
Compounded annual return over the last 15.175342465753424   years for QQQ using Ito formula is: 16.3757728066648 % per year.


As one can see the numbers are almost the same. There is a small difference from the actual numbers which is taken into account by the Brownian motion term. Now one can see why a triple levaraged ETF will have a volatility drag using Ito formula. A $3 \times $ levaraged ETF like SPXL, TQQQ will have $3 \times$ the mean return and the standard deviation as compared to SPY, QQQ. Lets make sure that this is correct.

In [65]:
#Fetch the historical prices for SPXL and TQQQ and compute the mean and standard deviation of them.

spxl_info = yfinance.Ticker("SPXL")
tqqq_info = yfinance.Ticker("TQQQ")
spxl_history = spxl_info.history(start = start_date, end = end_date)
tqqq_history = tqqq_info.history(start = start_date, end = end_date)

dailyret_spxl = spxl_history.loc[:, "Close"].pct_change()
dailyret_tqqq = tqqq_history.loc[:,"Close"].pct_change()

#Compute the average return for SPXL and TQQQ
mean_spxl = np.mean(dailyret_spxl)
mean_tqqq = np.mean(dailyret_tqqq)

#Since we computed the daily returns based on trading days, to annualize the returns we multiply by 252, for the trading days 
print("Average annual return over the last", number_of_years, " years for SPXL is:", mean_spxl*252*100,"% per year.")
print("Average annual return over the last", number_of_years, " years for TQQQ is:", mean_tqqq*252*100,"% per year.")


#Compute the standard deviation of returns and then annualize them.

sigma_spxl = np.std(dailyret_spxl)*np.sqrt(252)
sigma_tqqq = np.std(dailyret_tqqq)*np.sqrt(252)

print("Annual volatility over last last", number_of_years, " for SPXL is:", sigma_spxl*100,"% per year.")
print("Annual volatility over last", number_of_years, "  years for TQQQ is:", sigma_tqqq*100,"% per year.")

Average annual return over the last 15.175342465753424  years for SPXL is: 36.806397087463274 % per year.
Average annual return over the last 15.175342465753424  years for TQQQ is: 50.66050645765304 % per year.
Annual volatility over last last 15.175342465753424  for SPXL is: 51.99041605221223 % per year.
Annual volatility over last 15.175342465753424   years for TQQQ is: 61.623688397722596 % per year.


In [66]:
#Compounded returns for SPXL and TQQQ.

compunded_spxl = np.log(spxl_history["Close"][len(spxl_history)-1]/spxl_history["Close"][0])/number_of_years
compunded_tqqq = np.log(tqqq_history["Close"][len(tqqq_history)-1]/tqqq_history["Close"][0])/number_of_years

print("Compounded annual return over the last", number_of_years, "  years for SPXL is:", compunded_spxl*100, "% per year.")
print("Compounded annual return over the last", number_of_years, "  years for TQQQ is:", compunded_tqqq*100, "% per year.")

Compounded annual return over the last 15.175342465753424   years for SPXL is: 23.00955485874493 % per year.
Compounded annual return over the last 15.175342465753424   years for TQQQ is: 31.35759311837374 % per year.


In [67]:
#Now one can compute the compounded returns using Ito formula as we saw above. 
compounded_spxl_ito = mean_spxl*252 - ((sigma_spxl*sigma_spxl)/2)
compounded_tqqq_ito = mean_tqqq*252-((sigma_tqqq*sigma_tqqq)/2)

print("Compounded annual return over the last", number_of_years, "  years for SPXL using Ito formula is:", compounded_spxl_ito*100, "% per year.")
print("Compounded annual return over the last", number_of_years, "  years for TQQQ using Ito formula is:", compounded_tqqq_ito*100, "% per year.")

Compounded annual return over the last 15.175342465753424   years for SPXL using Ito formula is: 23.291380281052643 % per year.
Compounded annual return over the last 15.175342465753424   years for TQQQ using Ito formula is: 31.67311159895499 % per year.


Looking at the numbers above will give an illusion that these ETFs are safe to invest over long term. Nonetheless, these ETFs are not suitable for long term holdings. Using actual data we can quantify the drag associated with these ETFs. Using this we can extrapolate the performance of these ETFs to longer holding periods. As we will see even though data available for these ETFs may suggest that they are safe for longer term holding, the extrapolation of the data will show the problems with this argument.

In [68]:
#Do the analysis for SPY and QQQ over the wider data range.

#Get the data for date range of extrapolation
start_date_extrapolation = "1999-03-10"
end_date_extrapolation = "2025-04-13"

number_of_years_extrapolation = (dt.datetime.strptime(end_date_extrapolation,"%Y-%m-%d") - dt.datetime.strptime(start_date_extrapolation, "%Y-%m-%d"))/timedelta(days=365)
                 
spy_history_extrapolation = spy_info.history(start = start_date_extrapolation , end = end_date_extrapolation )
qqq_history_extrapolation = qqq_info.history(start = start_date_extrapolation, end = end_date_extrapolation)

spy_history_extrapolation.reset_index(inplace = True)
qqq_history_extrapolation.reset_index(inplace = True)

#Compute the mean returns, volatility and compounded returns 

dailyret_spy = spy_history_extrapolation.loc[:, "Close"].pct_change()
dailyret_qqq = qqq_history_extrapolation.loc[:,"Close"].pct_change()

#Compute the average return for SPY and QQQ
mean_spy = np.mean(dailyret_spy)
mean_qqq = np.mean(dailyret_qqq)

#Since we computed the daily returns based on trading days, to annualize the returns we multiply by 252, for the trading days 
print("Average annual return over the last ", number_of_years_extrapolation, "years for SPY is:", mean_spy*252*100,"%")
print("Average annual return over the last ", number_of_years_extrapolation," years for QQQ is:", mean_qqq*252*100,"%")

#Compute the standard deviation of returns and then annualize them.

sigma_spy = np.std(dailyret_spy)*np.sqrt(252)
sigma_qqq = np.std(dailyret_qqq)*np.sqrt(252)

#Usually the standard deviation is annualized and shown as percentage. This is also called the volatility of the stock.

print("Annual volatility over last", number_of_years_extrapolation, " years for SPY is:", sigma_spy*100,"% per year.")
print("Annual volatility over last", number_of_years_extrapolation, "  years for QQQ is:", sigma_qqq*100,"% per year.")

#computing the annualized compound interest over the 15 years of data.
compunded_spy = np.log(spy_history["Close"][len(spy_history)-1]/spy_history["Close"][0])/number_of_years_extrapolation
compunded_qqq = np.log(qqq_history["Close"][len(qqq_history)-1]/qqq_history["Close"][0])/number_of_years_extrapolation

print("Compounded annual return over the last", number_of_years_extrapolation, " years for SPY is:", compunded_spy*100, "% per year.")
print("Compounded annual return over the last", number_of_years_extrapolation, " years for QQQ is:", compunded_qqq*100, "% per year.")



Average annual return over the last  26.112328767123287 years for SPY is: 9.12958615137576 %
Average annual return over the last  26.112328767123287  years for QQQ is: 12.741412031767343 %
Annual volatility over last 26.112328767123287  years for SPY is: 19.4843769464844 % per year.
Annual volatility over last 26.112328767123287   years for QQQ is: 27.27630867600866 % per year.
Compounded annual return over the last 26.112328767123287  years for SPY is: 7.186011159119585 % per year.
Compounded annual return over the last 26.112328767123287  years for QQQ is: 9.48607991940333 % per year.


In [69]:
#Now we use the extrapolation to compute the compounded returns. Assuming the relationship observed between average returns and volatility for SPXL and TQQQ hold
# 
compounded_spxl_ito = (2.67*mean_spy)*252 - ((9*sigma_spy*sigma_spy)/2)
compounded_tqqq_ito = (2.75*mean_qqq)*252-((9*sigma_qqq*sigma_qqq)/2)

print("Compounded annual return over the last", number_of_years_extrapolation, "  years for SPXL using Ito formula is:", compounded_spxl_ito*100, "% per year.")
print("Compounded annual return over the last", number_of_years_extrapolation, "  years for TQQQ using Ito formula is:", compounded_tqqq_ito*100, "% per year.")

Compounded annual return over the last 26.112328767123287   years for SPXL using Ito formula is: 7.292152499502108 % per year.
Compounded annual return over the last 26.112328767123287   years for TQQQ using Ito formula is: 1.5590174128594592 % per year.



<h1>Conclusion</h1>
<ul>
  <li>As one can see using Ito formula we can mathematically explain the nature of compounded returns of levaraged ETFs. </li>
  <li>If one extrapolates the returns for SPXL and TQQQ using Ito's formula we can see that since 1999 the compunded annual return for the levaraged ETFs are even lower than their unlevaraged counterparts.</li>

</ul> 
